In [2]:
from IPython.display import display, HTML

import sys

from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.mllib.clustering import BisectingKMeans, BisectingKMeansModel
import matplotlib.pyplot as plt
import numpy
import geopandas
import pandas
import geopy
import seaborn as sns

In [3]:
conf = SparkConf().setAppName("HackTheMachineHierarchical")
conf = conf.setMaster("local[*]")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [4]:
sc

<SparkContext master=local[*] appName=HackTheMachineHierarchical>

In [5]:
df = sqlContext.read\
                .format("csv")\
                .options(header="true", inferSchema="true")\
                .load("/srv/scratch/hackthemachine/data/AIS_ASCII_by_UTM_Month/2015/AIS_2015_01_Zone10.csv")

In [6]:
from pyspark.sql.functions import when, lit, col
from pyspark.sql.types import DoubleType, LongType
clust_df = df.select("MMSI", "LAT", "LON", "BaseDateTime")
clust_df = clust_df.withColumn("BaseDateTime", clust_df.BaseDateTime.cast(LongType()))
clust_df = clust_df.withColumn("MMSI", clust_df.MMSI.cast(LongType()))

In [7]:
clust_df.rdd.take(5)

[Row(MMSI=367013050, LAT=47.64809, LON=-122.35087, BaseDateTime=1420088417),
 Row(MMSI=367436210, LAT=46.19725, LON=-123.80085, BaseDateTime=1420088537),
 Row(MMSI=367492890, LAT=47.82257, LON=-122.4704, BaseDateTime=1420088548),
 Row(MMSI=367350050, LAT=37.8169, LON=-122.40924, BaseDateTime=1420088569),
 Row(MMSI=316013431, LAT=48.41467, LON=-123.38626, BaseDateTime=1420088613)]

In [ ]:
from pyspark.mllib.linalg.distributed import RowMatrix, IndexedRow, IndexedRowMatrix
from pyspark.mllib.linalg import Vectors
clust_df_vec = clust_df.rdd.map(lambda d: IndexedRow(d[0], Vectors.dense(d[1], d[2], d[3])))
clust_df_rm = IndexedRowMatrix(clust_df_vec).toBlockMatrix().transpose().toIndexedRowMatrix()

In [ ]:
dimsum_sim = clust_df_rm.columnSimilarities(0.1)

In [ ]:
sim_entries = dimsum_sim.entries

In [ ]:
sim_entries.take(5)

In [ ]:
sim_entries.count()

In [ ]:
dimsum_sim.toIndexedRowMatrix().rows.take(10)